In [ ]:
import re

In [ ]:
sensors, beacons = [], []
with open("day15_input.txt") as file:
    for line in file:
        matches = re.findall("x=(-?\d+), y=(-?\d+)", line)
        sensors.append(tuple(map(int, matches[0])))
        beacons.append(tuple(map(int, matches[1])))

# Part 1

In [ ]:
y = 2_000_000
impossible = set()
for sensor, beacon in zip(sensors, beacons):
    sx, sy = sensor
    bx, by = beacon
    dist = abs(sx - bx) + abs(sy - by)
    rest = dist - abs(y - sy)
    if rest > 0:
        impossible.update(range(sx - rest, sx + rest + 1))
        if by == y:
            impossible.remove(bx)

len(impossible)

# Part 2

In [ ]:
from itertools import combinations

In [ ]:
def get_outside_lines(sensor, beacon):
    sx, sy = sensor
    bx, by = beacon
    dist = abs(sx - bx) + abs(sy - by)
    top = (sx, sy - dist - 1)
    bot = (sx, sy + dist + 1)
    lft = (sx - dist - 1, sy)
    rgt = (sx + dist + 1, sy)
    return [(lft, top), (lft, bot), (top, rgt), (bot, rgt)]

In [ ]:
def intersect_between(line1, line2):
    (x1, y1), (x2, y2) = line1
    (x3, y3), (x4, y4) = line2
    denom = (y4 - y3) * (x2 - x1) - (x4 - x3) * (y2 - y1)
    if denom == 0:  # parallel
        return None
    ua = ((x4 - x3) * (y1 - y3) - (y4 - y3) * (x1 - x3)) / denom
    if ua < 0 or ua > 1:  # out of range
        return None
    ub = ((x2 - x1) * (y1 - y3) - (y2 - y1) * (x1 - x3)) / denom
    if ub < 0 or ub > 1:  # out of range
        return None
    x = x1 + ua * (x2 - x1)
    y = y1 + ua * (y2 - y1)
    return (x, y)

In [ ]:
def point_outside(sensor, beacon, point):
    sx, sy = sensor
    bx, by = beacon
    px, py = point
    dist_b = abs(sx - bx) + abs(sy - by)
    dist_p = abs(sx - px) + abs(sy - py)
    return dist_p > dist_b

In [ ]:
bb_max = 4_000_000
bb = [
    [(0, 0), (0, bb_max)],
    [(0, 0), (bb_max, 0)],
    [(bb_max, 0), (bb_max, bb_max)],
    [(0, bb_max), (bb_max, bb_max)],
]
lines = bb + [
    line
    for sensor, beacon in zip(sensors, beacons)
    for line in get_outside_lines(sensor, beacon)
]

print(f"There's {len(lines)} lines to check, with possibly {len(list(combinations(lines, 2)))} intersections")

In [ ]:
intersections = set()
for line1, line2 in combinations(lines, 2):
    if intersect := intersect_between(line1, line2):
        x, y = map(int, intersect)
        if (0 <= x <= bb_max) and (0 <= y <= bb_max):
            intersections.add((x, y))

print(f"There's only {len(intersections)} intersections of interest")

In [ ]:
for intersect in intersections:
    if all(
        point_outside(sensor, beacon, intersect)
        for sensor, beacon in zip(sensors, beacons)
    ):
        print(intersect, "is outside all ranges")